***
# from ztfquery import fritz
***

### ztfquery.query enables you to get information from the ZTF-II "fritz marshal". 
It corresponds to alert and assocated informations.

ztfquery.fritz basically is a python wrapper of the fritz version of the skyportal web API (https://docs.fritz.science/api.html)

**Note**: you will need a token from you fritz account (https://fritz.science/profile)


**This tutorial will focus on Sample access and user accounts **, 

- see Fritz 2.2 for storing and retrieving data.
- see Fritz 2.1 for individual target data (lightcurve, alerts, spectra, source information)



You will be able to download, for a given target, its:

    - Samples (i.e. 'sources')
    - user account
    - groups

You will use for that `fritz.download_{this}` with `this` being is any of these. 
You can either get the data as they are in Frizt, or use dedicated `Fritz{Object}` that have useful methods. 

***
# Step 0: imports

In [1]:
%matplotlib notebook
from ztfquery import fritz

***
# Fritz Groups


Fritz organise `source`s in groups, like `"Same host SNe"`, each group has its own `id`. The `FritzGroup` object enables you to see all the existing groups and, most importantly, to go from `id` to `groupname` using the `FritzGroup.fetch_groupname(id)` and `FritzGroup.fetch_groupid(group_name)` class method.

As usual the `download_groups()` enables to get the group object. In practice, you don't really need that. 

The `FritzGroups.load()` is similar to the usual `.from_name()` class method. It dowloads the groups data and store them in `$ZTFDATA/sample/fritz_groups.json` such that next time you call the function is it will be much faster. use `force_dl=True` to update the file

In [14]:
group = fritz.FritzGroups.load() 

In [15]:
group.data

,modified,nickname,id,private,single_user_group,name,created_at
0,2020-11-30T08:49:10.252369,acai,217,False,False,ACAI,2020-11-30T08:49:10.252369
1,2020-12-03T02:34:16.003443,acai_tns,226,False,False,ACAI_TNS,2020-12-03T02:34:16.003443
2,2020-10-28T00:52:37.590023,agnflare,71,False,False,AGN Flares,2020-10-28T00:52:37.590023
3,2021-03-04T04:15:45.629195,AMCVn,256,False,False,AMCVn,2021-03-04T04:15:45.629195
4,2020-10-25T21:19:31.751195,AGt1,63,False,False,AmpelGroup_test1,2020-10-25T21:19:31.751195
...,...,...,...,...,...,...,...
79,2020-10-21T06:20:34.588728,young,52,False,False,Young Type Ia Supernovae,2020-10-21T06:20:34.588728
80,2020-11-24T20:05:58.713932,associates,213,False,False,ZTF II Associates,2020-11-24T20:05:58.713932
81,2020-10-21T06:20:34.665079,nuc,58,False,False,ZTFBH Nuclear,2020-10-21T06:20:34.665079
82,2020-10-26T16:33:05.875241,ZTFReST,64,False,False,ZTFReST,2020-10-26T16:33:05.875241


the `group.groupid_to_groupname()` and `group.groupname_to_groupid()` enables yuse to do the conversion once the group object is loaded. 

Again, you are invited to use `fetch_groupname(groupid)` this looks for the `groupid` in the `groups.data["id"]` and returns the corresponding name. If none are found, `fetch_groupname` update the group data by redownloading it and retries

#### Example,  get the id of the "Same host SNe"

In [16]:
fritz.FritzGroups.fetch_groupid("Same host SNe")

251

In [17]:
fritz.FritzGroups.fetch_groupname(251)

'Same host SNe'

***
# Sample

You can download a group `sample` corresponding to the list of individual `source` from this group using the `fritz.download_sample()` function. It contains the usual `get_object`, `store` options but also many option top filter the data queried to fritz. (see also `fritz.download_sources()`)


The `FritzSample` object is a collection of Sources and contains the following useful attributes:
- `sources`: list of FritzSource 
- `nsource`: number of sources
- `names`: names of the sources
- `data`: DataFrame containing a summary of the indivudual source information (like ra, dec, redshift, classification)
- `groupid` and `groupname`: name and ID of the sample group is any

and then has the usual I/O methods:
- `store(id_)`: store the sources into `$ZTFDATA/sample/fritz_sample_{id_}.json` where id_ is by default `groupid` 
- `get_source(name)`: returns the `FritzSource` associated to the given name
- `to_json` and `read_json`: save or load the sources dict anywhere 


But most likely the methods you are going to use the most are:

- `FritzSample.from_group()`: loads the object given a groupname or a groupid (similar to the `from_name()` classmethod)
- `fetch_data()`: bulk download, in parallel the data assocated to your sample, could be `lightcurve`, `spectra`, `alerts` (or source but this is useless)


`fetch_data()` will basically run the `.from_name()` of the individual lightcurve, spectra etc. It then has access to the same tricks as before, except if `force_dl=True`, it downloads only the object not stored yet. By default, `fetch_data()` store the object it downloads.

`fetch_data()` uses the `fritz.bulk_download()` function

In [31]:
%time cosmo_sources = fritz.download_sample(groupid=251, get_object=True)

CPU times: user 69.7 ms, sys: 23.3 ms, total: 93 ms
Wall time: 7.37 s


In [23]:
cosmo_sources.store()

In [25]:
%time s = fritz.FritzSample.from_group("Same host SNe")

CPU times: user 22.4 ms, sys: 3.01 ms, total: 25.4 ms
Wall time: 28.4 ms


In [26]:
s.data

,redshift,ra,dec,classification,created_at,last_detected_at
name,,,,,,
ZTF17aadlxmv,0.062000,127.448018,33.906536,Ia,2020-11-04T21:26:32.647533,2020-11-27T12:55:52.000326+00:00
ZTF18aagrcfl,0.047961,236.095888,39.558108,Ia,2021-02-08T14:06:57.239913,2021-03-18T11:51:39.000947+00:00
ZTF18aahmxqa,NaN,276.703544,51.140438,[],2020-11-16T16:59:22.138630,2021-03-18T12:17:38.996147+00:00
ZTF18aakaljn,0.070000,145.294241,24.022435,Ia,2020-11-04T21:18:24.552628,2021-02-10T08:53:13.997777+00:00
ZTF18aakecej,0.066450,203.773313,53.875161,Ia,2020-11-04T21:05:53.292080,2021-03-18T08:43:39.999361+00:00
...,...,...,...,...,...,...
ZTF21aajfpwk,0.079139,151.912872,58.213053,Ia,2021-02-09T09:52:04.590116,2021-03-19T05:33:45.999364+00:00
ZTF21aakjfmr,0.031162,256.723990,30.272275,[],2021-02-11T13:04:38.764182,2021-03-02T12:24:34.001299+00:00
ZTF21aaldrdx,0.017475,49.474492,-7.297525,[],2021-02-17T03:43:29.568716,2021-02-17T03:34:29.003503+00:00


In [27]:
lc = s.fetch_data("lightcurve", show_progress=True)

FloatProgress(value=0.0)

### Check you `$ZTFDATA/fritz/lightcurve directory`

In [32]:
import os

In [35]:
os.listdir(os.path.join(fritz.FRITZSOURCE,"lightcurve"))

['fritz_lightcurve_ZTF18aahmxqa.csv',
 'fritz_lightcurve_ZTF19abpyqog.csv',
 'fritz_lightcurve_ZTF18adachwf.csv',
 'fritz_lightcurve_ZTF20aaqvzko.csv',
 'fritz_lightcurve_ZTF19acnwelq.csv',
 'fritz_lightcurve_ZTF18aamdfcw.csv',
 'fritz_lightcurve_ZTF20abptxls.csv',
 'fritz_lightcurve_ZTF18abnucig.csv',
 'fritz_lightcurve_ZTF19aatzlmw.csv',
 'fritz_lightcurve_ZTF20abydkrl.csv',
 'fritz_lightcurve_ZTF20abhjesj.csv',
 'fritz_lightcurve_ZTF19acdtmwh.csv',
 'fritz_lightcurve_ZTF21aajfpwk.csv',
 'fritz_lightcurve_ZTF19aacxwfb.csv',
 'fritz_lightcurve_ZTF20aaelulu.csv',
 'fritz_lightcurve_ZTF18aawmvbj.csv',
 'fritz_lightcurve_ZTF18abdffeo.csv',
 'fritz_lightcurve_ZTF20aavpwxl.csv',
 'fritz_lightcurve_ZTF20aagnbpw.csv',
 'fritz_lightcurve_ZTF21aaaollj.csv',
 'fritz_lightcurve_ZTF20acebweq.csv',
 'fritz_lightcurve_ZTF20acpwjus.csv',
 'fritz_lightcurve_ZTF19aamhgwm.csv',
 'fritz_lightcurve_ZTF19aaloezs.csv',
 'fritz_lightcurve_ZTF19abkfqqp.csv',
 'fritz_lightcurve_ZTF19aambfxc.csv',
 'fritz_ligh

### And the returned dictionary

In [36]:
lc

{'ZTF17aadlxmv': <ztfquery.fritz.FritzPhotometry at 0x7fb4455afcd0>,
 'ZTF18aagrcfl': <ztfquery.fritz.FritzPhotometry at 0x7fb445594a30>,
 'ZTF18aahmxqa': <ztfquery.fritz.FritzPhotometry at 0x7fb445594ac0>,
 'ZTF18aakaljn': <ztfquery.fritz.FritzPhotometry at 0x7fb446017670>,
 'ZTF18aakecej': <ztfquery.fritz.FritzPhotometry at 0x7fb4455af730>,
 'ZTF18aamdfcw': <ztfquery.fritz.FritzPhotometry at 0x7fb445d23130>,
 'ZTF18aaqcozd': <ztfquery.fritz.FritzPhotometry at 0x7fb445d233d0>,
 'ZTF18aasdted': <ztfquery.fritz.FritzPhotometry at 0x7fb445d23490>,
 'ZTF18aawmvbj': <ztfquery.fritz.FritzPhotometry at 0x7fb445d6b970>,
 'ZTF18aazgfkq': <ztfquery.fritz.FritzPhotometry at 0x7fb445d23a90>,
 'ZTF18abdbysy': <ztfquery.fritz.FritzPhotometry at 0x7fb4455af1f0>,
 'ZTF18abdffeo': <ztfquery.fritz.FritzPhotometry at 0x7fb4455affd0>,
 'ZTF18abdgwvs': <ztfquery.fritz.FritzPhotometry at 0x7fb445d23970>,
 'ZTF18abdmgab': <ztfquery.fritz.FritzPhotometry at 0x7fb4455943a0>,
 'ZTF18abktzep': <ztfquery.fritz.F

In [37]:
lightcurve_of_one = lc["ZTF18abdgwvs"]

In [38]:
lightcurve_of_one.data

,obj_id,ra,dec,filter,mjd,instrument_id,instrument_name,ra_unc,dec_unc,origin,id,groups,mag,magerr,magsys,limiting_mag
0,ZTF18abdgwvs,NaN,NaN,ztfg,58269.317095,1,ZTF,NaN,NaN,NaN,18193753,"[251, 30]",NaN,NaN,ab,19.208401
1,ZTF18abdgwvs,NaN,NaN,ztfr,58269.356528,1,ZTF,NaN,NaN,NaN,18193754,"[251, 30]",NaN,NaN,ab,18.867800
2,ZTF18abdgwvs,NaN,NaN,ztfr,58272.332917,1,ZTF,NaN,NaN,NaN,18193755,"[251, 30]",NaN,NaN,ab,20.041300
3,ZTF18abdgwvs,NaN,NaN,ztfg,58272.378843,1,ZTF,NaN,NaN,NaN,18193756,"[251, 30]",NaN,NaN,ab,20.158001
4,ZTF18abdgwvs,NaN,NaN,ztfr,58275.338623,1,ZTF,NaN,NaN,NaN,18193757,"[251, 30]",NaN,NaN,ab,20.526600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ZTF18abdgwvs,NaN,NaN,ztfr,59137.101435,1,ZTF,NaN,NaN,NaN,18193855,"[251, 30]",NaN,NaN,ab,20.424700
103,ZTF18abdgwvs,267.021763,23.488074,ztfg,59137.134294,1,ZTF,NaN,NaN,NaN,18193856,"[251, 30]",20.000799,0.200151,ab,20.500500
104,ZTF18abdgwvs,267.021727,23.487998,ztfg,59137.150509,1,ZTF,NaN,NaN,NaN,18193857,"[251, 30]",19.891500,0.207019,ab,20.332600
105,ZTF18abdgwvs,NaN,NaN,ztfr,59139.092419,1,ZTF,NaN,NaN,NaN,18193858,"[251, 30]",NaN,NaN,ab,20.126200


In [ ]:
lightcurve_of_one.